In [1]:
import cv2
import os

def generate_dataset():
    # Load the Haar Cascade for face detection
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    def face_cropped(img):
        # Check if the image is valid
        if img is None:
            return None

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        # Check if no faces are detected using len(faces) instead of .size
        if len(faces) == 0:  # Corrected condition to check if no faces are detected
            return None
        
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face
     
    # Create the 'data' directory if it doesn't exist
    if not os.path.exists('data'):
        os.makedirs('data')

    # Use the default camera index 0
    cap = cv2.VideoCapture(0) 
    if not cap.isOpened():
        print("Error: Could not open video capture.")
        return

    # Input ID for the user
    id = input("Enter your ID: ")  # User input for ID, convert to int if necessary
    try:
        id = int(id)
    except ValueError:
        print("Invalid ID entered, using default ID 1.")
        id = 1

    img_id = 0
     
    while True:
        ret, frame = cap.read()
        
        # Check if frame capture was successful
        if not ret or frame is None:
            print("Error: Failed to capture image.")
            break

        cropped_face = face_cropped(frame)
        if cropped_face is not None:
            img_id += 1
            face = cv2.resize(cropped_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            # Define the directory where you want to save the images
            save_directory = r"C:\Users\mohap\OneDrive\Desktop\Face Recognizer data"

            # Ensure the directory exists (create it if it doesn't)
            if not os.path.exists(save_directory):
                os.makedirs(save_directory)

            # Construct the file name using user id and image id
            file_name_path = os.path.join(save_directory, f"user.{id}.{img_id}.jpg")

            # Print the absolute path for confirmation (optional)
            print(f"Saving image to: {os.path.abspath(file_name_path)}")

            # Save the image
            cv2.imwrite(file_name_path, face)

            # Optionally, display the face image and annotate it with the image id
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped Face", face)
             
        # Exit loop on pressing Enter or capturing 200 images
        if cv2.waitKey(1) == 13 or img_id == 200:
            break
             
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed...")

# Call the function to generate dataset
generate_dataset()

Enter your ID:  1


Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.1.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.2.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.3.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.4.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.5.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.6.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.7.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.8.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.9.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.10.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.11.jpg
Saving image to: C:\Users\mohap\OneDrive\Desktop\Face Recognizer data\user.1.12.jpg
S

In [5]:
import os
import cv2
from PIL import Image  # pip install pillow
import numpy as np     # pip install numpy
# Create the 'data' directory if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

 
def train_classifier(data_dir):
    # Check if the directory exists
    if not os.path.exists(data_dir):
        print(f"Error: Directory '{data_dir}' does not exist.")
        return

    # Load image paths
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
     
    faces = []
    ids = []
     
    for image in path:
        img = Image.open(image).convert('L')  # Convert to grayscale
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
         
        faces.append(imageNp)
        ids.append(id)
         
    ids = np.array(ids)
     
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    print("Training complete and classifier saved as 'classifier.xml'")

train_classifier("data")


Training complete and classifier saved as 'classifier.xml'


In [2]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
import mysql.connector

window = tk.Tk()
window.title("Face Recognition System")

# Name, Age, Address fields in the GUI
l1 = tk.Label(window, text="Name", font=("Algerian", 20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window, text="Age", font=("Algerian", 20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window, text="Address", font=("Algerian", 20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

# Train classifier function
def train_classifier():
    data_dir = r"C:\Users\mohap\OneDrive\Desktop\Face Recognizer data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)
    
    ids = np.array(ids)

    # Train the classifier and save it
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Result', 'Training dataset completed!!!')

# Training button
b1 = tk.Button(window, text="Training", font=("Algerian", 20), bg='orange', fg='red', command=train_classifier)
b1.grid(column=0, row=4)

# Detect face function
def detect_face():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)

        for (x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            id, pred = clf.predict(gray_image[y:y + h, x:x + w])
            confidence = int(100 * (1 - pred / 300))

            # Connect to the database and fetch the name
            mydb = mysql.connector.connect(
                host="localhost",
                user="root",
                passwd="",
                database="Authorized_user"
            )
            mycursor = mydb.cursor()
            mycursor.execute("SELECT name FROM my_table WHERE id = %s", (id,))
            result = mycursor.fetchone()

            if result:
                name = result[0]  # Extract the name from the tuple
            else:
                name = "Unknown"  # Handle the case where the name is not found

            # Display the name and confidence
            if confidence > 74:
                cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else:
                cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

        return img

    def recognize(img, clf, faceCascade):
        img = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
        return img

    # Load face detection classifier
    faceCascade = cv2.CascadeClassifier("C:/Users/mohap/Downloads/haarcascade_frontalface_default.xml")

    # Load the trained classifier for face recognition
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    # Start video capture
    video_capture = cv2.VideoCapture(0)

    if not video_capture.isOpened():
        messagebox.showerror("Error", "Failed to access the camera.")
        return

    while True:
        ret, img = video_capture.read()
        if not ret:
            break
        
        img = recognize(img, clf, faceCascade)
        cv2.imshow("Face Detection", img)

        # Exit loop when Enter is pressed
        if cv2.waitKey(1) == 13:  # 13 is the ASCII code for Enter key
            break

    # Release video capture and close windows
    video_capture.release()
    cv2.destroyAllWindows()

# Detect face button
b2 = tk.Button(window, text="Detect the Face", font=("Algerian", 20), bg='green', fg='white', command=detect_face)
b2.grid(column=1, row=4)

# Generate dataset function
def generate_dataset():
    if t1.get() == "" or t2.get() == "" or t3.get() == "":
        messagebox.showinfo('Result', 'Please provide complete details of the user')
    else:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="",
            database="Authorized_user"
        )
        mycursor = mydb.cursor()
        mycursor.execute("SELECT * FROM my_table")
        myresult = mycursor.fetchall()
        id = len(myresult) + 1  # Automatically increment the ID
        
        # Insert the user details into the database
        sql = "INSERT INTO my_table (id, Name, Age, Address) VALUES (%s, %s, %s, %s)"
        val = (id, t1.get(), t2.get(), t3.get())
        mycursor.execute(sql, val)
        mydb.commit()

        # Start capturing the face images for the dataset
        face_classifier_path = "haarcascade_frontalface_default.xml"

        # Check if the classifier file exists
        if not os.path.exists(face_classifier_path):
            messagebox.showerror("Error", "Haar Cascade Classifier file not found!")
            return

        face_classifier = cv2.CascadeClassifier(face_classifier_path)

        def face_cropped(img):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)

            if faces == ():
                return None
            for (x, y, w, h) in faces:
                cropped_face = img[y:y + h, x:x + w]
            return cropped_face

        # Open the camera
        cap = cv2.VideoCapture(0)

        if not cap.isOpened():
            messagebox.showerror("Error", "Failed to access the camera.")
            return

        img_id = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if face_cropped(frame) is not None:
                img_id += 1
                face = cv2.resize(face_cropped(frame), (200, 200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = f"data/user.{id}.{img_id}.jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Cropped Face", face)

                if cv2.waitKey(1) == 13 or int(img_id) == 200:  # Press Enter to stop or after capturing 200 images
                    break

        # Release the camera and close any open windows after capturing dataset
        cap.release()
        cv2.destroyAllWindows()

        messagebox.showinfo('Result', 'Generating dataset completed!!!')


# Generate dataset button
b3 = tk.Button(window, text="Generate Dataset", font=("Algerian", 20), bg='pink', fg='black', command=generate_dataset)
b3.grid(column=2, row=4)

# Set window size
window.geometry("800x200")

# Start the Tkinter main loop
window.mainloop()


In [3]:
import cv2
import numpy as np
import mysql.connector

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)

        # Predict the ID of the face
        id, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))

        # Fetch the name from the database based on the predicted ID
        name = "UNKNOWN"  # Default name if not found in database
        if confidence > 70:
            # Connect to the MySQL database
            mydb = mysql.connector.connect(
                host="localhost",
                user="root",  # Replace with your MySQL username
                passwd="",     # Replace with your MySQL password
                database="Authorized_user"  # Replace with your database name
            )
            mycursor = mydb.cursor()

            # Query the database for the name associated with the ID
            mycursor.execute("SELECT name FROM my_table WHERE id = %s", (id,))
            result = mycursor.fetchone()

            if result:
                name = result[0]  # Extract the name from the result

            # Close the database connection
            mycursor.close()
            mydb.close()

        # Display the name based on the confidence and id
        if confidence > 70:
            cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

    return img

# Load the Haar Cascade Classifier for face detection
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load the pre-trained face recognizer classifier
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("C:/Users/mohap/ML case study/classifier.xml")

# Open the video capture
video_capture = cv2.VideoCapture(0)  # Use DirectShow backend (try cv2.CAP_ANY if 0 doesn't work)

if not video_capture.isOpened():
    print("Error: Could not open video capture.")
    exit()

while True:
    ret, img = video_capture.read()

    if not ret:
        print("Failed to grab frame.")
        continue

    # Call the face detection function
    img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)

    # Show the resulting frame
    cv2.imshow("Face Detection", img)

    # Exit the loop when the Enter key is pressed
    if cv2.waitKey(1) == 13:  # 13 is the ASCII code for Enter key
        break

# Release the video capture and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()
